In [ ]:
import matplotlib.pyplot as plt
from numpy import sin, cos, pi, inf, sqrt, exp
import random
import numpy as np
from scipy.integrate import quad
import sympy as sp


In [ ]:
def draw_signal(t, signal_t, label):
    fig, ax = plt.subplots()
    plt.ylabel("Signal")
    plt.xlabel('Time')
    plt.title(label)
    plt.grid(True)
    ax.plot(t, signal_t)

In [ ]:
dt = 0.01
time = np.arange(-100, 100, dt)
y = lambda t: sin(t)
yt = [y(t) for t in time]

In [ ]:
draw_signal(time, yt, "")

In [ ]:
a = .1
y = lambda t: sin(t) + a * (random.uniform(0, 1) - 0.5)
yt = [y(t) for t in time]

In [ ]:
draw_signal(time, yt, "")

In [ ]:
def find_num_derivative_array(yt, dt):
    return [(yt[k + 1] - yt[k]) / dt for k in range(len(yt) - 1)]

In [ ]:
# test dericative calculation on a simple function
fig, ax = plt.subplots(2)
test_time = np.arange(-6.28, 6.28, dt)
test_yt = [cos(ts) for ts in test_time]
test_yt_d = find_num_derivative_array([sin(ts) for ts in test_time], dt)
ax[0].plot(test_time, test_yt)
ax[1].plot(test_time[:-1], test_yt_d)
None

In [ ]:
y_derivative = find_num_derivative_array(yt, dt)
draw_signal(time[:-1], y_derivative, "Derivative")

In [ ]:
def find_num_derivative_lambda(time, y, dt):
    return [(y(time[k + 1]) - y(time[k])) / dt for k in range(len(time) - 1)]

In [ ]:
def trapz_ft(function, time, dt):
    def integrand(nu):
        integrate = lambda t: function(t) * np.exp(-2j * np.pi * nu * t)
        temp = [integrate(i) for i in time]
        return np.trapz(temp, time)
    return lambda nu1: integrand(nu1)

In [ ]:
def trapz_ft_derivative(function, time, dt):
    def integrand(nu):
        integrate = lambda t: function(t) * np.exp(-2j * np.pi * nu * t) * 2j * pi * nu 
        temp = [integrate(i) for i in time]
        return np.trapz(temp, time)
    return lambda nu: integrand(nu)

In [ ]:
# Define symbols for time and frequency
t, nu = sp.symbols('t nu')

# Example symbolic expression
expr = sp.sin(t**2)

# Convert the symbolic expression to a Numpy function
func = sp.lambdify([t], expr, "numpy")

def trapz_ft_derivative_improved(function, time, dt):
    """Improved version of the Fourier Transform Derivative using Trapezoidal Rule."""

    def integrand(nu):
        omega = 2 * np.pi * nu
        exponential_term = np.exp(-1j * omega * time)

        # Avoid expensive elementwise multiplication between arrays
        exp_term_real = np.cos(-omega * time)
        exp_term_imag = np.sin(-omega * time)
        product = function(time) * exponential_term

        # Vectorized operation using real and imaginary parts
        integral_real = np.trapz((product[:, None] * exp_term_real).sum(axis=0))
        integral_imag = np.trapz((product[:, None] * exp_term_imag).sum(axis=0))

        return integral_real + 1j * integral_imag

    return integrand

In [ ]:
short_time = np.linspace(-1, 1, 5)
y_derivative_ft = trapz_ft_derivative_improved(y, time, 1)
print(y_derivative_ft)

In [ ]:
fig, ax = plt.subplots()
ax.grid(True)

ax.plot(short_time, [y_derivative_ft(i) for i in short_time])
# ax.legend()
plt.show()
